Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.


In [1]:
import pandas as pd
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.tail(10)

490                                  Lab: B12 969 2007\n
491                                   )and 8mo in 2009\n
492    .Moved to USA in 1986. Suffered from malnutrit...
493                                              r1978\n
494    . Went to Emerson, in Newfane Alaska. Started ...
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
dtype: object

### There are four types of time format 
1) xx/xx/xx    MM/DD/YY
2) xx/xx       MM/YY
3) (xx) Jan/Feb/Mar.... YY
4) xx           YY 

I compare the time by using the formula YY+Month/12+Day/365 

In [2]:
def date_sorter():
    
    import numpy 
    from time import strptime 
    
    def s_split(s):  # type 1 split 
        for string in s:
            if string.find('-') != -1 :
                s_split=string.split('-')
            else:    
                s_split=string.split('/')
            
            year=float(s_split[2])
            if year < 100:
                year = 1900+year  

            day=float(s_split[1])
            month=float(s_split[0])
            c=year+day/365.0+month/12.0
        return c 

    def s_split_2(s): # type 2 split 
        for string in s:
            if string.find('-') != -1 :
                s_split=string.split('-')
            else:    
                s_split=string.split('/')

            year=float(s_split[1])
            if year < 100:
                year = 1900+year  

            day=float(1)
            month=float(s_split[0])
            c=year+day/365.0+month/12.0
      
        return c

    def s_split_3(s):
        for string in s:
            string=string.replace(',','')
            string=string.replace('.','')
            s_split=string.split(' ')
        
            if len(s_split)==3:
                year=(s_split[2])
            
                if s_split[1].isalpha():
                    month=s_split[1]  
                    day=s_split[0]
                else:
                    month=s_split[0]
                    day=s_split[1]
                
            elif len(s_split)==2:
                year=s_split[1]
                month=s_split[0]  
                day=(1.0)
        #==========Typo Error========================
            if month == 'Janaury':
                month ='January'
            if month == 'Decemeber':
                month = 'Dec'
        #=============================================       
            for fmt in ('%B', '%b'):   # convert the names of month to numbers 
                try:
                    month_n=strptime(month,fmt).tm_mon
                except ValueError:
                    pass       
            c=float(year)+float(day)/365.0+(month_n)/12.0
        return c     
    

    format1=df.str.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}')
    format1[72]=[format1[72][0]]  # manually change one case 
    df1=df[format1.str.len() == 0]
    ct1=format1[format1.str.len() != 0].apply(s_split)
    
    format2=df1.str.findall(r'\d{1,2}[/]\d{2,4}')
    df2=df1[format2.str.len() == 0]
    ct2=format2[format2.str.len() != 0].apply(s_split_2)
    
    # find the data format with (xx)Jan/Feb...[,.] YY. 
    format3=df2.str.findall(r'(?:\d{2} )*?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.,]* (?:\d{2}[, ]*)?\d{4}')
    df3=df2[format3.str.len() == 0]
    ct3=format3[format3.str.len() != 0].apply(s_split_3)
    
    format4=df3.str.findall(r'\d{4}') 
    df4=df3[format4.str.len() == 0]
    ct4=format4.apply(lambda x: float(x[0])+1.0/12.0+1.0/365)
    
    #combine all the data 
    correct_time=pd.concat([ct1,ct3,ct2,ct4],ignore_index=True)
    
    index=numpy.argsort(correct_time)
    sort=sorted(range(500), key=lambda k: index[k])
    sorted_date=sorted(range(500), key=lambda k: sort[k])
    return pd.Series(sorted_date)
    
#sum(format1.str.len() == 0) # 375 left not fit in this case 